# Posthoc test

## Bonferroni

In [11]:
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
import pandas as pd

# create a dataset
Web = np.array([32, 30, 25, 24, 36])
Field_sales = np.array([42, 20, 19, 40, 30])
Dummy = np.array([20, 25, 30, 24, 23])
tdata = pd.DataFrame({'Web campaign':Web, 'Filed Sales':Field_sales, 'Nothing to be done':Dummy,},index=[1,2,3,4,5])
tdata 

,Web campaign,Filed Sales,Nothing to be done
1,32,42,20
2,30,20,25
3,25,19,30
4,24,40,24
5,36,30,23


In [27]:
import itertools
import scipy.stats as st

# create all column pairs in the dataset 
def column_pairs(df):
  lis = df.columns
  pairs = []
  for pair in itertools.combinations(lis, 2):
      pairs.append(pair)
  return pairs

pairs = column_pairs(tdata)

# bonferroni_correction: it isn't recommened to use bonferroni_correction if the number of columns in a dataset is over five.
def bonferroni_correction(pairs,p_value=.05):
  statistical_significance = p_value
  bonferroni_correction = statistical_significance/len(pairs)
  return bonferroni_correction

bonferroni_correction = bonferroni_correction(pairs)

"""
Levene test : equal_variance test 
→ p-value < 0.05 then t-test Welch's method
→ p-value >= 0.05 then t-test Student's method
"""
# equal_variance_test(levene) and ttest
def levene_ttest(df,pairs,p_value):
  ttest_pvalue = []
  for p in pairs:
    levene_p = st.levene(df[p[0]],df[p[1]],center='mean')
    if levene_p[1] < p_value:
      t,ttest_p = st.ttest_ind(df[p[0]],df[p[1]],equal_var=False) # Welch
      ttest_pvalue.append(ttest_p)
    else:
      t,ttest_p = st.ttest_ind(df[p[0]],df[p[1]],equal_var=True) # Student
      ttest_pvalue.append(ttest_p)

  result_dict = {}
  for k,v in zip(pairs,ttest_pvalue):
    result_dict[k] = v

  return result_dict

levene_ttest(tdata,pairs,bonferroni_correction)



{('Filed Sales', 'Nothing to be done'): 0.2873958965134085,
 ('Web campaign', 'Filed Sales'): 0.8839820611631122,
 ('Web campaign', 'Nothing to be done'): 0.10767480820917394}